In [33]:
from azureml.core import Workspace, Experiment
import pandas as pd

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-129725
Azure region: southcentralus
Subscription id: de47103e-2da6-4f5e-88fc-d18b27fd249b
Resource group: aml-quickstarts-129725


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster

cpu_cluster_name = 'cpucluster'

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


### YOUR CODE HERE ###

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [41]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C" : uniform(0.1,1),
                            "--max_iter": choice(50,100,150,200)})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1,evaluation_interval = 1,delay_evaluation=5)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./training",entry_script="train.py",compute_target=cpu_cluster_name)
### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =  HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    max_total_runs=100,
    max_concurrent_runs=4,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=policy,
    estimator=est
)

### YOUR CODE HERE ###

In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

experiment=Experiment(ws,ws.name)
hiperdrive_run = experiment.submit(hyperdrive_config)

RunDetails(hiperdrive_run).show()
### YOUR CODE HERE ###

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [23]:
import joblib
# Get your best run and save the model from that run.
best_run = hiperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength: ',parameter_values[1])
print('\n Max iterations: ',parameter_values[3])



### YOUR CODE HERE ###

Best Run Id:  HD_5580f750-89f0-4654-9eba-f40886891fcc_93

 Accuracy: 0.9103692463328276

 Regularization Strength:  0.5453675819604022

 Max iterations:  150


In [25]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
from azureml.core import Workspace, Dataset



dataset = Dataset.get_by_name(ws, name='Bank-marketing')
ds = dataset.to_pandas_dataframe()

### YOUR CODE HERE ###

In [45]:
from training.train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

df = x.join(y)

In [50]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=df,
    label_column_name='y',
    n_cross_validations=5)

In [51]:
# Submit your automl run

local_run = experiment.submit(automl_config, show_output = True)
### YOUR CODE HERE ###

Running on local machine
Parent Run ID: AutoML_ba008d9a-71f1-414d-9939-627afc2ba1ef

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.


In [ ]:
displayHTML("<a href={} target='_blank'>Azure Portal: {}</a>".format(local_run.get_portal_url(), local_run.id))

In [ ]:
# Retrieve and save your best automl model.
best_run, fitted_model = local_run.get_output()


### YOUR CODE HERE ###